<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Create-pipeline-components-to-use-in-SpaCy-NLP-pipeline" data-toc-modified-id="Create-pipeline-components-to-use-in-SpaCy-NLP-pipeline-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Create pipeline components to use in SpaCy NLP pipeline</a></span><ul class="toc-item"><li><span><a href="#Class-to-tag-employee-nouns-as-entities" data-toc-modified-id="Class-to-tag-employee-nouns-as-entities-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Class to tag employee nouns as entities</a></span></li><li><span><a href="#YearMatcher:-Add-custom-_.is_year-attribute-to-year-tokens-that-are-part-of-Date-entities" data-toc-modified-id="YearMatcher:-Add-custom-_.is_year-attribute-to-year-tokens-that-are-part-of-Date-entities-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>YearMatcher: Add custom <code>_.is_year</code> attribute to year tokens that are part of Date entities</a></span></li></ul></li><li><span><a href="#Code-for-relationship-extraction-and-helper-functions" data-toc-modified-id="Code-for-relationship-extraction-and-helper-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Code for relationship extraction and helper functions</a></span><ul class="toc-item"><li><span><a href="#Function:-extract_emp_relations" data-toc-modified-id="Function:-extract_emp_relations-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Function: extract_emp_relations</a></span></li><li><span><a href="#Testing-handling-of-tuples-(sorting-to-match-years-with-cardinal-numbers)" data-toc-modified-id="Testing-handling-of-tuples-(sorting-to-match-years-with-cardinal-numbers)-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Testing handling of tuples (sorting to match years with cardinal numbers)</a></span></li><li><span><a href="#Function:-print_doc_info" data-toc-modified-id="Function:-print_doc_info-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Function: print_doc_info</a></span></li></ul></li><li><span><a href="#Define-sentence-structure-types" data-toc-modified-id="Define-sentence-structure-types-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define sentence structure types</a></span><ul class="toc-item"><li><span><a href="#Company_noun-in-subject,--Emp_num-and-EMP_NOUN-in-predicate" data-toc-modified-id="Company_noun-in-subject,--Emp_num-and-EMP_NOUN-in-predicate-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span><code>Company_noun</code> in subject,  <code>Emp_num</code> and <code>EMP_NOUN</code> in predicate</a></span><ul class="toc-item"><li><span><a href="#[Company]--[was|-had]-Emp_Num-EMP_NOUN" data-toc-modified-id="[Company]--[was|-had]-Emp_Num-EMP_NOUN-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>[Company]  [<code>was</code>| <code>had</code>] <code>Emp_Num</code> <code>EMP_NOUN</code></a></span></li><li><span><a href="#we-[employed-|-have]-Emp_num-EMP_NOUN" data-toc-modified-id="we-[employed-|-have]-Emp_num-EMP_NOUN-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span><code>we</code> [<code>employed</code> | <code>have</code>] <code>Emp_num</code> <code>EMP_NOUN</code></a></span></li><li><span><a href="#Emp_num-below-EMP_NOUN-in-parse-tree,-org_tok-left-of-verb" data-toc-modified-id="Emp_num-below-EMP_NOUN-in-parse-tree,-org_tok-left-of-verb-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span><code>Emp_num</code> below <code>EMP_NOUN</code> in parse tree, <code>org_tok</code> left of verb</a></span></li><li><span><a href="#Org_tok-[employment_noun]-was-Emp_Num-EMP_NOUN" data-toc-modified-id="Org_tok-[employment_noun]-was-Emp_Num-EMP_NOUN-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span><code>Org_tok</code> [employment_noun] <code>was</code> <code>Emp_Num</code> <code>EMP_NOUN</code></a></span></li><li><span><a href="#Org_tok-that-is-not-the-company" data-toc-modified-id="Org_tok-that-is-not-the-company-3.1.5"><span class="toc-item-num">3.1.5&nbsp;&nbsp;</span><code>Org_tok</code> that is not the company</a></span></li></ul></li><li><span><a href="#Emp_noun-and-Company_noun--in-subject,-Emp_num-in-predicate" data-toc-modified-id="Emp_noun-and-Company_noun--in-subject,-Emp_num-in-predicate-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span><code>Emp_noun</code> and <code>Company_noun</code>  in subject, <code>Emp_num</code> in predicate</a></span><ul class="toc-item"><li><span><a href="#EMP_NOUN-Company_noun--was-(compount-predicate)--Emp_Num-and-Emp_Num" data-toc-modified-id="EMP_NOUN-Company_noun--was-(compount-predicate)--Emp_Num-and-Emp_Num-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span><code>EMP_NOUN</code> <code>Company_noun</code>  <code>was</code> (compount predicate)  <code>Emp_Num</code> and <code>Emp_Num</code></a></span></li></ul></li><li><span><a href="#Emp_num-above-and-below-EMP_NOUN-in-parse-tree" data-toc-modified-id="Emp_num-above-and-below-EMP_NOUN-in-parse-tree-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span><code>Emp_num</code> above and below <code>EMP_NOUN</code> in parse tree</a></span></li><li><span><a href="#Testing-on-sentences" data-toc-modified-id="Testing-on-sentences-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Testing on sentences</a></span></li></ul></li><li><span><a href="#Testing-with-paragraphs" data-toc-modified-id="Testing-with-paragraphs-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Testing with paragraphs</a></span></li><li><span><a href="#Matcher-testing" data-toc-modified-id="Matcher-testing-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Matcher testing</a></span></li></ul></div>

In [1]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher, PhraseMatcher
from spacy.tokens import Doc, Span, Token
import pandas as pd
import re

from path import Path, getcwdu

from IPython.display import display, HTML

In [2]:
subset_df = pd.read_excel('../data/subset_employee_count_paragraphs.xlsx')

In [3]:
nonum_pat = re.compile(r"^([^\d]|[$][\d]+)[^\d]*$", re.I)

train_df = pd.read_csv('data/classifier_input_train.csv', index_col=0)
train_df = train_df[train_df.len > 19][train_df.para_text.apply(lambda x: not re.match(nonum_pat, x))]
val_df = pd.read_csv('data/classifier_input_val.csv', index_col=0)
val_df = val_df[val_df.len > 19][val_df.para_text.apply(lambda x: not re.match(nonum_pat, x))]

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


## Create pipeline components to use in SpaCy NLP pipeline

### Class to tag employee nouns as entities

In [4]:
# Templated from: https://spacy.io/usage/processing-pipelines#custom-components 
class EmpNounRecognizer(object):
    """A spaCy v2.0 pipeline component that sets entity annotations
    based on list of terms. Terms are labelled as EMP_NOUN. Additionally,
    ._.has_emp_noun and ._.is_emp_noun is set on the Doc/Span and Token
    respectively."""
    name = 'employee_nouns'  # component name, will show up in the pipeline

    def __init__(self, nlp, terms=tuple(), label='EMP_NOUN'):
        """Initialise the pipeline component. The shared nlp instance is used
        to initialise the matcher with the shared vocab, get the label ID and
        generate Doc objects as phrase match patterns.
        """
        self.label = nlp.vocab.strings[label]  # get entity label ID

        # Set up the PhraseMatcher – it can now take Doc objects as patterns,
        # so even if the list of terms is long, it's very efficient
        patterns = [nlp(term) for term in terms]
        self.matcher = PhraseMatcher(nlp.vocab)
        self.matcher.add('EMP_NOUN', None, *patterns)

        # Register attribute on the Token. We'll be overwriting this based on
        # the matches, so we're only setting a default value, not a getter.
        Token.set_extension('is_emp_noun', default=False)

        # Register attributes on Doc and Span via a getter that checks if one of
        # the contained tokens is set to is_emp_noun == True.
        Doc.set_extension('has_emp_noun', getter=self.has_emp_noun)
        Span.set_extension('has_emp_noun', getter=self.has_emp_noun)

    def __call__(self, doc):
        """Apply the pipeline component on a Doc object and modify it if matches
        are found. Return the Doc, so it can be processed by the next component
        in the pipeline, if available.
        """
        matches = self.matcher(doc)
        spans = []  # keep the spans for later so we can merge them afterwards
        for _, start, end in matches:
            # Generate Span representing the entity & set label
            entity = Span(doc, start, end, label=self.label)
            spans.append(entity)
            # Set custom attribute on each token of the entity
            for token in entity:
                token._.set('is_emp_noun', True)
            # Overwrite doc.ents and add entity – be careful not to replace!
            doc.ents = list(doc.ents) + [entity]
        for span in spans:
            # Iterate over all spans and merge them into one token. This is done
            # after setting the entities – otherwise, it would cause mismatched
            # indices!
            span.merge()
        return doc  # don't forget to return the Doc!

    def has_emp_noun(self, tokens):
        """Getter for Doc and Span attributes. Returns True if one of the tokens
        is an employee noun. Since the getter is only called when we access the
        attribute, we can refer to the Token's 'is_emp_noun' attribute here,
        which is already set in the processing step."""
        return any([t._.get('is_emp_noun') for t in tokens])

In [5]:
emp_terms_list = ["associates", "employees", "individuals", "people", "persons", "team members"]
emp_terms_list = emp_terms_list + [s.title() for s in emp_terms_list] + [s.upper() for s in emp_terms_list]

In [6]:
nlp = spacy.load('en_core_web_lg')

In [7]:
emp_noun_recognizer = EmpNounRecognizer(nlp, emp_terms_list)

In [8]:
nlp.add_pipe(emp_noun_recognizer, last=True) 

In [9]:
print('Pipeline', nlp.pipe_names) 

Pipeline ['tagger', 'parser', 'ner', 'employee_nouns']


### YearMatcher: Add custom `_.is_year` attribute to year tokens that are part of Date entities

In [10]:
class YearMatcher(object):
    name = 'year_matcher'
    
    def __init__(self, nlp, pattern_list, match_id='Year'):
        # register a new token extension to flag year tokens
        Token.set_extension('is_year', default=False)
        self.matcher = Matcher(nlp.vocab)
        self.matcher.add(match_id, None, pattern_list)

    def __call__(self, doc):
        matches = self.matcher(doc)
        spans = []  # collect the matched spans here
        for match_id, start, end in matches:
            spans.append(doc[start:end])
        for span in spans:
            span.merge()   # merge
            for token in span:
                token._.is_year = True  # mark token as a year
        return doc

year_patterns = [{'ENT_TYPE': 'DATE', 'TAG' : 'CD', 'SHAPE' : 'dddd'}]

In [11]:
year_matcher = YearMatcher(nlp, year_patterns)

nlp.add_pipe(year_matcher, last=True) 

In [12]:
print('Pipeline', nlp.pipe_names) 

Pipeline ['tagger', 'parser', 'ner', 'employee_nouns', 'year_matcher']


## Code for relationship extraction and helper functions

### Function: extract_emp_relations

In [13]:
def extract_emp_relations(doc):
    relations = []
    org_ents = [e for e in doc.ents if e.label_ == 'ORG']
    for word in filter(lambda w: w.ent_type_ == 'EMP_NOUN', doc):
        if org_ents:
                relations.append((org_ents[0], word))
        if word.dep_ in ('attr', 'dobj'):
            num_tok = [w for w in word.children if w.dep_ == 'nummod']
            if word.head.dep_ == 'ROOT':
                subject = [w for w in word.head.lefts if w.dep_ == 'nsubj']
            else:
                subject = [w for w in word.head.lefts if w.dep_ == 'nsubj']
            if subject:
                subject = subject[0]
                #[print(str(p) + '  :  ' + str(p.dep_)) for p in subject.subtree]
                sub_poss = [p for p in subject.subtree if p.dep_ == 'poss']
                if sub_poss:
                    sub_poss = sub_poss[0]
                    relations.append((sub_poss, word))
                relations.append((subject, word))
            if num_tok:
                num_tok = num_tok[0]
                relations.append((num_tok, word))
        elif word.dep_ == 'pobj' and word.head.dep_ == 'prep':
            subject = word.head.head 
            num_toks = [c for c in word.children if c.dep_ == 'nummod' and c.ent_type_ == 'CARDINAL']
            if num_toks:
                relations.append((num_toks[0], word))
            elif word.head.head.head.pos_ == 'VERB':
                cards = [c for c in word.head.head.head.rights if c.tag_== 'CD' and c.ent_type_ == 'CARDINAL']
                dates = [e for e in word.doc.ents if e.label_ == 'DATE']
                if cards:
                    relations.append((cards[0], word))
            relations.append((subject, word))
        elif word.dep_ == 'conj':
            child_num_toks = [w for w in word.children if w.dep_ == 'nummod'] 
            head_num_tok = [w for w in [word.head] if w.tag_ == 'CD']
            if child_num_toks and head_num_tok:
                years = [(y.i, y) for y in word.doc if y._.is_year == True]
                emp_counts = [(c.i, c) for c in child_num_toks + head_num_tok]
                order_indices = [years.index(y) for y in sorted(years, reverse=True, key = lambda x: x[1])]
                year_emps = [(years[i][1], emp_counts[i][1]) for i in order_indices]
                relations.append((max(year_emps)[1], word))
            else: 
                relations.append((child_num_toks[0], word))
    return relations


### Testing handling of tuples (sorting to match years with cardinal numbers)

In [14]:
iyears =[(13, 2015), (11, 2016)]

In [15]:
iemps = [(15, '43,246'), (12, '53,146')]

In [16]:
sorted(iemps)

[(12, '53,146'), (15, '43,246')]

In [17]:
year_sorted = sorted(iyears, key = lambda tup: tup[1], reverse=True)

In [18]:
year_sorted

[(11, 2016), (13, 2015)]

In [19]:
order_indices = [iyears.index(x) for x in year_sorted]

In [20]:
order_indices

[1, 0]

In [21]:
year_emps = [(iyears[i][1], iemps[i][1]) for i in order_indices]

In [22]:
year_emps

[(2016, '53,146'), (2015, '43,246')]

In [23]:
max(year_emps)[1]

'53,146'

### Function: print_doc_info 

In [24]:
def make_tok_df(doc, tok_filter_func=False):
    """Return a dataframe showing attributes for each token in doc."""
    
    if tok_filter_func:
        toks = list(filter(tok_filter_func, doc))
    else:
        toks = doc
    doc_dict = {'tok_ent' : [tok.ent_type_ for tok in toks], 
        'toks' : [tok for tok in toks], 
        'dep' : [tok.dep_ for tok in toks], 
        'head' : [tok.head for tok in toks], 
        'head_dep' : [tok.head.dep_ for tok in toks], 
        'dep_def' : [spacy.explain(tok.dep_) for tok in toks],
        'pos' : [tok.pos_ for tok in toks], 
        'tag' : [tok.tag_ for tok in toks], 
        'tag_def' : [spacy.explain(tok.tag_) for tok in toks], 
               }
    columns = [ 'tok_ent', 'toks', 'dep', 'head', 'head_dep', 'pos', 'tag',  'dep_def', 'tag_def' ]
    return pd.DataFrame(doc_dict, columns=columns)

def make_span_df(doc, entities=True):
    """Return df showing attributes for each entity or noun chunk in doc."""
    
    columns = ['tok_i', 'entity', 'ent_label', 'root', 'root_dep','dep_def', 
               'root_head',  'root_head_dep', 'root_head_pos'  ]
    if entities:
        target_spans = doc.ents
        df_name = 'doc_entities'
    else: 
        target_spans = list(doc.noun_chunks)
        df_name = 'doc_noun_chunks'
    doc_dict = {'tok_i' : [e.start for e in target_spans], 
        'entity' : [e.text for e in target_spans], 
        'ent_label' : [e.label_ for e in target_spans], 
        'root' : [e.root.text for e in target_spans], 
        'root_dep' : [e.root.dep_ for e in target_spans], 
        'dep_def' : [spacy.explain(e.root.dep_) for e in target_spans],
        'root_head' : [e.root.head for e in target_spans], 
        'root_head_dep' : [e.root.head.dep_ for e in target_spans], 
        'root_head_pos' : [e.root.head.pos_ for e in target_spans]}
    try :
        df = pd.DataFrame(doc_dict, columns=columns) 
        df.name = df_name
        if not entities:
            df.columns = ['tok_i', 'noun_chunk', 'NP', 'root', 'root_dep','dep_def',
                          'root_head',  'root_head_dep', 'root_head_pos' ]
    except: 
        df = [(k, doc_dict[k]) for k in doc_dict.keys()]
    return df

def print_df(df, print_df_name=True):
    """Display rendered HTML version of df in Jupyter notebook"""
    
    if print_df_name and hasattr(df, 'name'):
        print("DataFrame is named " + str(df.name))
    display(HTML(df.to_html()))


def print_doc_info(doc):
    """Print dfs for doc entities, noun_chunks, and CARDINAL toks"""
    
    print("doc is: ")
    print(doc)
    print('-' * 50)
    print("Entities are: ")
    print_df(make_span_df(doc), print_df_name=False)
    print('-' * 50)
    print("Noun chunks are: ")
    print_df(make_span_df(doc, entities=False), print_df_name=False)
    print('-' * 50)
    print("Cardinal entities are: ")
    card_filter = lambda w: w.ent_type_ == 'CARDINAL'
    print_df(make_tok_df(doc, card_filter), print_df_name=False)

## Define sentence structure types

### `Company_noun` in subject,  `Emp_num` and `EMP_NOUN` in predicate

#### [Company]  [`was`| `had`] `Emp_Num` `EMP_NOUN`

Examples: 


`"At December 31, 2016, Bio-Rad had approximately 8,250 employees."`

Desired output: 

`(Bio-Rad, had, 8,250, employees)`

In [25]:
ex1 = nlp('At December 31, 2016, Bio-Rad had approximately 8,250 employees.')

In [26]:
displacy.render(ex1, style='ent', jupyter=True, options={'distance': 90})

In [27]:
displacy.render(ex1, style='dep', jupyter=True, options={'distance': 90})

In [28]:
print_df(make_tok_df(ex1))

,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,,At,prep,had,ROOT,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
1,DATE,December,pobj,At,prep,PROPN,NNP,object of preposition,"noun, proper singular"
2,DATE,31,nummod,December,pobj,NUM,CD,None,cardinal number
3,DATE,",",punct,December,pobj,PUNCT,",",punctuation,"punctuation mark, comma"
4,DATE,2016,nummod,December,pobj,NUM,CD,None,cardinal number
5,,",",punct,had,ROOT,PUNCT,",",punctuation,"punctuation mark, comma"
6,ORG,Bio,compound,Rad,nsubj,PROPN,NNP,None,"noun, proper singular"
7,ORG,-,punct,Rad,nsubj,PUNCT,HYPH,punctuation,"punctuation mark, hyphen"
8,ORG,Rad,nsubj,had,ROOT,PROPN,NNP,nominal subject,"noun, proper singular"
9,,had,ROOT,had,ROOT,VERB,VBD,None,"verb, past tense"


In [29]:
print_doc_info(ex1)

doc is: 
At December 31, 2016, Bio-Rad had approximately 8,250 employees.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,1,"December 31, 2016",DATE,December,pobj,object of preposition,At,prep,ADP
1,6,Bio-Rad,ORG,Rad,nsubj,nominal subject,had,ROOT,VERB
2,10,"approximately 8,250",CARDINAL,"8,250",nummod,None,employees,dobj,NOUN
3,12,employees,EMP_NOUN,employees,dobj,direct object,had,ROOT,VERB


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,NP,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,1,December,NP,December,pobj,object of preposition,At,prep,ADP
1,6,Bio-Rad,NP,Rad,nsubj,nominal subject,had,ROOT,VERB
2,10,"approximately 8,250 employees",NP,employees,dobj,direct object,had,ROOT,VERB


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,CARDINAL,approximately,advmod,"8,250",nummod,ADV,RB,adverbial modifier,adverb
1,CARDINAL,"8,250",nummod,employees,dobj,NUM,CD,None,cardinal number


In [30]:
ex1_emp_tok = ex1[12]

In [31]:
ex1_emp_tok.dep_

'dobj'

In [32]:
[x for x in ex1_emp_tok.children]

[8,250]

In [33]:
#[w for w in ex1_emp_tok.head.lefts if w.dep_ == 'nsubj']
print('Token head: ' + str(ex1_emp_tok.head) + '       head.dep_:'+ str(ex1_emp_tok.head.dep_) )
for w in ex1_emp_tok.head.lefts:
    print(str(w) + '       head.left.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        for b in w.subtree:
            print(str(b) + '       child.dep_:' + str(b.dep_) + '       child.head:' + str(b.head))
print('Token ancestors: ')
for w in ex1_emp_tok.ancestors:
    print(str(w) + '       ancestor.dep_:' + str(w.dep_))
print('Token children: ')
for w in ex1_emp_tok.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token head: had       head.dep_:ROOT
At       head.left.dep_:prep
,       head.left.dep_:punct
Rad       head.left.dep_:nsubj
Rad is nsubj. Subtree is:
Bio       child.dep_:compound       child.head:Rad
-       child.dep_:punct       child.head:Rad
Rad       child.dep_:nsubj       child.head:had
Token ancestors: 
had       ancestor.dep_:ROOT
Token children: 
8,250       child.dep_:nummod


In [34]:
extract_emp_relations(ex1)

[(Bio-Rad, employees), (Rad, employees), (8,250, employees)]

#### `we` [`employed` | `have`] `Emp_num` `EMP_NOUN`

`"As of January 31, 2017, we employed 7,683 individuals."`

Desired output: 

`(we, employed, 7,683, individuals)`

In [35]:
ex2 = nlp("As of January 31, 2017, we employed 7,683 individuals.")

In [36]:
print_df(make_tok_df(ex2))

,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,,As,prep,employed,ROOT,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
1,,of,prep,As,prep,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
2,DATE,January,pobj,of,prep,PROPN,NNP,object of preposition,"noun, proper singular"
3,DATE,31,nummod,January,pobj,NUM,CD,None,cardinal number
4,DATE,",",punct,January,pobj,PUNCT,",",punctuation,"punctuation mark, comma"
5,DATE,2017,nummod,January,pobj,NUM,CD,None,cardinal number
6,,",",punct,employed,ROOT,PUNCT,",",punctuation,"punctuation mark, comma"
7,,we,nsubj,employed,ROOT,PRON,PRP,nominal subject,"pronoun, personal"
8,,employed,ROOT,employed,ROOT,VERB,VBD,None,"verb, past tense"
9,CARDINAL,"7,683",nummod,individuals,dobj,NUM,CD,None,cardinal number


In [37]:
print_doc_info(ex2)

doc is: 
As of January 31, 2017, we employed 7,683 individuals.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,2,"January 31, 2017",DATE,January,pobj,object of preposition,of,prep,ADP
1,9,"7,683",CARDINAL,"7,683",nummod,None,individuals,dobj,NOUN
2,10,individuals,EMP_NOUN,individuals,dobj,direct object,employed,ROOT,VERB


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,NP,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,2,January,NP,January,pobj,object of preposition,of,prep,ADP
1,7,we,NP,we,nsubj,nominal subject,employed,ROOT,VERB
2,9,"7,683 individuals",NP,individuals,dobj,direct object,employed,ROOT,VERB


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,CARDINAL,"7,683",nummod,individuals,dobj,NUM,CD,None,cardinal number


In [38]:
ex2_emp_tok = ex2[10]

In [39]:
#[w for w in ex2_emp_tok.head.lefts if w.dep_ == 'nsubj']
print('Token dep_: ' + str(ex2_emp_tok.dep_))
print('Token head: ' + str(ex2_emp_tok.head) + '       head.dep_:'+ str(ex2_emp_tok.head.dep_) )
for w in ex2_emp_tok.head.lefts:
    print(str(w) + '       head.left.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        [print(x) for x in w.lefts]
        [print(s)for s in w.subtree if s.dep_ == 'poss']
print('Token ancestors: ')
for w in ex2_emp_tok.ancestors:
    print(str(w) + '       ancestor.dep_:' + str(w.dep_))
print('Token children: ')
for w in ex2_emp_tok.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token dep_: dobj
Token head: employed       head.dep_:ROOT
As       head.left.dep_:prep
,       head.left.dep_:punct
we       head.left.dep_:nsubj
we is nsubj. Subtree is:
Token ancestors: 
employed       ancestor.dep_:ROOT
Token children: 
7,683       child.dep_:nummod


In [40]:
extract_emp_relations(ex2)

[(we, individuals), (7,683, individuals)]

#### `Emp_num` below `EMP_NOUN` in parse tree, `org_tok` left of verb

Desired output: 

`(AEP, had, 17,634, employees)`

In [41]:
ex3 = nlp("As of December 31, 2016, the subsidiaries of AEP had a total of 17,634 employees.")

In [42]:
[tok for tok in ex3 if tok.ent_type_ == 'ORG']

[AEP]

In [43]:
print_df(make_tok_df(ex3))

,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,,As,prep,had,ROOT,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
1,,of,prep,As,prep,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
2,DATE,December,pobj,of,prep,PROPN,NNP,object of preposition,"noun, proper singular"
3,DATE,31,nummod,December,pobj,NUM,CD,None,cardinal number
4,DATE,",",punct,December,pobj,PUNCT,",",punctuation,"punctuation mark, comma"
5,DATE,2016,nummod,December,pobj,NUM,CD,None,cardinal number
6,,",",punct,had,ROOT,PUNCT,",",punctuation,"punctuation mark, comma"
7,,the,det,subsidiaries,nsubj,DET,DT,determiner,determiner
8,,subsidiaries,nsubj,had,ROOT,NOUN,NNS,nominal subject,"noun, plural"
9,,of,prep,subsidiaries,nsubj,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"


In [44]:
print_doc_info(ex3)

doc is: 
As of December 31, 2016, the subsidiaries of AEP had a total of 17,634 employees.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,2,"December 31, 2016",DATE,December,pobj,object of preposition,of,prep,ADP
1,10,AEP,ORG,AEP,pobj,object of preposition,of,prep,ADP
2,15,"17,634",CARDINAL,"17,634",nummod,None,employees,pobj,NOUN
3,16,employees,EMP_NOUN,employees,pobj,object of preposition,of,prep,ADP


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,NP,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,2,December,NP,December,pobj,object of preposition,of,prep,ADP
1,7,the subsidiaries,NP,subsidiaries,nsubj,nominal subject,had,ROOT,VERB
2,10,AEP,NP,AEP,pobj,object of preposition,of,prep,ADP
3,12,a total,NP,total,dobj,direct object,had,ROOT,VERB
4,15,"17,634 employees",NP,employees,pobj,object of preposition,of,prep,ADP


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,CARDINAL,"17,634",nummod,employees,pobj,NUM,CD,None,cardinal number


In [45]:
ex3_emp_tok = ex3[16]

ex3_emp_num_ent = ex3.ents[2]

In [46]:
ex3_emp_num_ent

17,634

In [47]:
ex3_emp_num_ent.root.ent_type_

'CARDINAL'

In [48]:
ex3[3].ent_type_

'DATE'

In [49]:
#[w for w in ex3_emp_tok.head.lefts if w.dep_ == 'nsubj']
print('Token dep_: ' + str(ex3_emp_tok.dep_))
print('Token head: ' + str(ex3_emp_tok.head) + '       head.dep_:'+ str(ex3_emp_tok.head.dep_) )
for w in ex3_emp_tok.head.lefts:
    print(str(w) + '       head.left.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        [print(x) for x in w.lefts]
        [print(s)for s in w.subtree if s.dep_ == 'poss']
print('Token ancestors: ')
for w in ex3_emp_tok.ancestors:
    print(str(w) + '       ancestor.dep_:' + str(w.dep_))
print('Token children: ')
for w in ex3_emp_tok.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token dep_: pobj
Token head: of       head.dep_:prep
Token ancestors: 
of       ancestor.dep_:prep
total       ancestor.dep_:dobj
had       ancestor.dep_:ROOT
Token children: 
17,634       child.dep_:nummod


In [50]:
displacy.render(ex3, style='dep', jupyter=True, options={'distance': 110})

In [51]:
print(ex3_emp_tok.head)
print(ex3_emp_tok.head.dep_)
print(ex3_emp_tok.head.head)
print(ex3_emp_tok.head.head.dep_)
print(ex3_emp_tok.head.head.head)
print(ex3_emp_tok.head.head.head.dep_)
print(ex3_emp_tok.head.head.head.pos_)
print(ex3_emp_tok.head.head.head.tag_)
print([c for c in ex3_emp_tok.head.head.head.children])
print([c.dep_ for c in ex3_emp_tok.head.head.head.children])
print([c.tag_ for c in ex3_emp_tok.head.head.head.children])
print([r for r in ex3_emp_tok.head.head.head.subtree if r.tag_== 'CD'])
print([c.ent_iob_ for c in ex3_emp_tok.head.head.head.children])
print([e for e in ex3.ents])
print([e.label_ for e in ex3.ents])
print([e for e in ex3.ents if e.label_ == 'DATE'])

of
prep
total
dobj
had
ROOT
VERB
VBD
[As, ,, subsidiaries, total, .]
['prep', 'punct', 'nsubj', 'dobj', 'punct']
['IN', ',', 'NNS', 'NN', '.']
[31, 2016, 17,634]
['', '', '', '', '']
[December 31, 2016, AEP, 17,634, employees]
['DATE', 'ORG', 'CARDINAL', 'EMP_NOUN']
[December 31, 2016]


In [52]:
extract_emp_relations(ex3)

[(AEP, employees), (17,634, employees), (total, employees)]

#### `Org_tok` [employment_noun] `was` `Emp_Num` `EMP_NOUN`

`"Alcoa's total worldwide employment at the end of 2016 was approximately 14,000 employees in 15 countries."`

Desired output: 

`(employment, was, 14,000, employees)`

In [53]:
ex4 = nlp("Alcoa's total worldwide employment at the end of 2016 was approximately 14,000 employees in 15 countries.")

In [54]:
print_df(make_tok_df(ex4))

,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,ORG,Alcoa,poss,employment,nsubj,PROPN,NNP,possession modifier,"noun, proper singular"
1,,'s,case,Alcoa,poss,PART,POS,None,possessive ending
2,,total,amod,employment,nsubj,ADJ,JJ,adjectival modifier,adjective
3,,worldwide,amod,employment,nsubj,ADJ,JJ,adjectival modifier,adjective
4,,employment,nsubj,was,ROOT,NOUN,NN,nominal subject,"noun, singular or mass"
5,,at,prep,employment,nsubj,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
6,DATE,the,det,end,pobj,DET,DT,determiner,determiner
7,DATE,end,pobj,at,prep,NOUN,NN,object of preposition,"noun, singular or mass"
8,DATE,of,prep,end,pobj,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
9,DATE,2016,pobj,of,prep,NUM,CD,object of preposition,cardinal number


In [55]:
print_doc_info(ex4)

doc is: 
Alcoa's total worldwide employment at the end of 2016 was approximately 14,000 employees in 15 countries.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,0,Alcoa,ORG,Alcoa,poss,possession modifier,employment,nsubj,NOUN
1,6,the end of 2016,DATE,end,pobj,object of preposition,at,prep,ADP
2,11,"approximately 14,000",CARDINAL,"14,000",nummod,None,employees,attr,NOUN
3,13,employees,EMP_NOUN,employees,attr,attribute,was,ROOT,VERB
4,15,15,CARDINAL,15,nummod,None,countries,pobj,NOUN


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,NP,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,0,Alcoa's total worldwide employment,NP,employment,nsubj,nominal subject,was,ROOT,VERB
1,6,the end,NP,end,pobj,object of preposition,at,prep,ADP
2,11,"approximately 14,000 employees",NP,employees,attr,attribute,was,ROOT,VERB
3,15,15 countries,NP,countries,pobj,object of preposition,in,prep,ADP


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,CARDINAL,approximately,advmod,"14,000",nummod,ADV,RB,adverbial modifier,adverb
1,CARDINAL,"14,000",nummod,employees,attr,NUM,CD,None,cardinal number
2,CARDINAL,15,nummod,countries,pobj,NUM,CD,None,cardinal number


In [56]:
ex4_emp_tok = ex4[13]

In [57]:
#[w for w in ex4_emp_tok.head.lefts if w.dep_ == 'nsubj']
print('Token dep_: ' + str(ex4_emp_tok.dep_))
print('Token head: ' + str(ex4_emp_tok.head) + '       head.dep_:'+ str(ex4_emp_tok.head.dep_) )
for w in ex4_emp_tok.head.lefts:
    print(str(w) + '       head.left.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        [print(x) for x in w.lefts]
        [print(s)for s in w.subtree if s.dep_ == 'poss']
print('Token ancestors: ')
for w in ex4_emp_tok.ancestors:
    print(str(w) + '       ancestor.dep_:' + str(w.dep_))
print('Token children: ')
for w in ex4_emp_tok.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token dep_: attr
Token head: was       head.dep_:ROOT
employment       head.left.dep_:nsubj
employment is nsubj. Subtree is:
Alcoa
total
worldwide
Alcoa
Token ancestors: 
was       ancestor.dep_:ROOT
Token children: 
14,000       child.dep_:nummod
in       child.dep_:prep


In [58]:
displacy.render(ex4, style='dep', jupyter=True, options={'distance': 90})

In [59]:
extract_emp_relations(ex4)

[(Alcoa, employees),
 (Alcoa, employees),
 (employment, employees),
 (14,000, employees)]

#### `Org_tok` that is not the company

Desired output: 

`(Company, employed, 1,562, persons)`

In [60]:
ex5 = nlp("The Company and its subsidiaries employed 1,562 persons at December 31, 2016, 114 of whom are covered by a collective bargaining agreement with District 10 of the International Association of Machinists.")

In [61]:
print_df(make_tok_df(ex5))

,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,,The,det,Company,nsubj,DET,DT,determiner,determiner
1,,Company,nsubj,employed,ROOT,PROPN,NNP,nominal subject,"noun, proper singular"
2,,and,cc,Company,nsubj,CCONJ,CC,coordinating conjunction,"conjunction, coordinating"
3,,its,poss,subsidiaries,conj,ADJ,PRP$,possession modifier,"pronoun, possessive"
4,,subsidiaries,conj,Company,nsubj,NOUN,NNS,conjunct,"noun, plural"
5,,employed,ROOT,employed,ROOT,VERB,VBD,None,"verb, past tense"
6,CARDINAL,"1,562",nummod,persons,dobj,NUM,CD,None,cardinal number
7,EMP_NOUN,persons,dobj,employed,ROOT,NOUN,NNS,direct object,"noun, plural"
8,,at,prep,employed,ROOT,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
9,DATE,December,pobj,at,prep,PROPN,NNP,object of preposition,"noun, proper singular"


In [62]:
print_doc_info(ex5)

doc is: 
The Company and its subsidiaries employed 1,562 persons at December 31, 2016, 114 of whom are covered by a collective bargaining agreement with District 10 of the International Association of Machinists.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,6,"1,562",CARDINAL,"1,562",nummod,None,persons,dobj,NOUN
1,7,persons,EMP_NOUN,persons,dobj,direct object,employed,ROOT,VERB
2,9,"December 31, 2016",DATE,December,pobj,object of preposition,at,prep,ADP
3,14,114,CARDINAL,114,nsubjpass,nominal subject (passive),covered,relcl,VERB
4,25,District 10,FAC,District,pobj,object of preposition,with,prep,ADP
5,28,the International Association of Machinists,ORG,Association,pobj,object of preposition,of,prep,ADP


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,NP,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,0,The Company,NP,Company,nsubj,nominal subject,employed,ROOT,VERB
1,3,its subsidiaries,NP,subsidiaries,conj,conjunct,Company,nsubj,PROPN
2,6,"1,562 persons",NP,persons,dobj,direct object,employed,ROOT,VERB
3,9,December,NP,December,pobj,object of preposition,at,prep,ADP
4,16,whom,NP,whom,pobj,object of preposition,of,prep,ADP
5,20,a collective bargaining agreement,NP,agreement,pobj,object of preposition,by,agent,ADP
6,25,District,NP,District,pobj,object of preposition,with,prep,ADP
7,28,the International Association,NP,Association,pobj,object of preposition,of,prep,ADP
8,32,Machinists,NP,Machinists,pobj,object of preposition,of,prep,ADP


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,CARDINAL,"1,562",nummod,persons,dobj,NUM,CD,None,cardinal number
1,CARDINAL,114,nsubjpass,covered,relcl,NUM,CD,nominal subject (passive),cardinal number


In [63]:
ex5_emp_tok = ex5[7]

In [64]:
ex5_emp_tok.dep_

'dobj'

In [65]:
displacy.render(ex5, style='dep', jupyter=True, options={'distance': 110})

In [66]:
print(ex5_emp_tok.head)
print(ex5_emp_tok.head.dep_)
print([c for c in ex5_emp_tok.head.lefts])
print([c.dep_ for c in ex5_emp_tok.head.head.head.children])
print([c.tag_ for c in ex5_emp_tok.head.head.head.children])
print([r for r in ex5_emp_tok.head.head.head.rights if r.tag_== 'CD'])
print([c.ent_iob_ for c in ex5_emp_tok.head.head.head.children])
print([e for e in ex5.ents])
print([e.label_ for e in ex5.ents])
print([e for e in ex5.ents if e.label_ == 'DATE'])

employed
ROOT
[Company]
['nsubj', 'dobj', 'prep', 'punct']
['NNP', 'NNS', 'IN', '.']
[]
['', 'B', '', '']
[1,562, persons, December 31, 2016, 114, District 10, the International Association of Machinists]
['CARDINAL', 'EMP_NOUN', 'DATE', 'CARDINAL', 'FAC', 'ORG']
[December 31, 2016]


In [67]:
extract_emp_relations(ex5)

[(the International Association of Machinists, persons),
 (its, persons),
 (Company, persons),
 (1,562, persons)]

### `Emp_noun` and `Company_noun`  in subject, `Emp_num` in predicate

#### `EMP_NOUN` `Company_noun`  `was` (compount predicate)  `Emp_Num` and `Emp_Num`

Examples:

`"The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31, 2015."`

Desired output: 

`(employees, Company, was, 31,800)`

In [68]:
ex6 = nlp("The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31, 2015.")

In [69]:
print_df(make_tok_df(ex6))

,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,,The,det,number,nsubj,DET,DT,determiner,determiner
1,,number,nsubj,was,ROOT,NOUN,NN,nominal subject,"noun, singular or mass"
2,,of,prep,number,nsubj,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
3,,full,amod,time,compound,ADJ,JJ,adjectival modifier,adjective
4,,-,punct,time,compound,PUNCT,HYPH,punctuation,"punctuation mark, hyphen"
5,,time,compound,employees,pobj,NOUN,NN,None,"noun, singular or mass"
6,EMP_NOUN,employees,pobj,of,prep,NOUN,NNS,object of preposition,"noun, plural"
7,,of,prep,employees,pobj,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
8,,the,det,Company,pobj,DET,DT,determiner,determiner
9,ORG,Company,pobj,of,prep,PROPN,NNP,object of preposition,"noun, proper singular"


In [70]:
print_doc_info(ex6)

doc is: 
The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31, 2015.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,6,employees,EMP_NOUN,employees,pobj,object of preposition,of,prep,ADP
1,9,Company,ORG,Company,pobj,object of preposition,of,prep,ADP
2,11,"approximately 31,800",CARDINAL,"31,800",attr,attribute,was,ROOT,VERB
3,14,"December 31, 2016",DATE,December,pobj,object of preposition,at,prep,ADP
4,19,"32,300",CARDINAL,"32,300",conj,conjunct,was,ROOT,VERB
5,21,"December 31, 2015",DATE,December,pobj,object of preposition,at,prep,ADP


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,NP,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,0,The number,NP,number,nsubj,nominal subject,was,ROOT,VERB
1,3,full-time employees,NP,employees,pobj,object of preposition,of,prep,ADP
2,8,the Company,NP,Company,pobj,object of preposition,of,prep,ADP
3,14,December,NP,December,pobj,object of preposition,at,prep,ADP
4,21,December,NP,December,pobj,object of preposition,at,prep,ADP


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,CARDINAL,approximately,advmod,"31,800",attr,ADV,RB,adverbial modifier,adverb
1,CARDINAL,"31,800",attr,was,ROOT,NUM,CD,attribute,cardinal number
2,CARDINAL,"32,300",conj,was,ROOT,NUM,CD,conjunct,cardinal number


In [71]:
ex6_emp_tok = ex6[6]

In [72]:
ex6_emp_tok.dep_

'pobj'

In [73]:
print(ex6_emp_tok.head.dep_)
print(ex6_emp_tok.head.head)
print(ex6_emp_tok.head.head.dep_)
print(ex6_emp_tok.head.head.head)
print(ex6_emp_tok.head.head.head.dep_)
print(ex6_emp_tok.head.head.head.pos_)
print(ex6_emp_tok.head.head.head.tag_)
print([c for c in ex6_emp_tok.head.head.head.children])
print([c.dep_ for c in ex6_emp_tok.head.head.head.children])
print([c.tag_ for c in ex6_emp_tok.head.head.head.children])
print([r for r in ex6_emp_tok.head.head.head.rights if r.tag_== 'CD'])
print([c.ent_iob_ for c in ex6_emp_tok.head.head.head.children])
print([e for e in ex6.ents])
print([e.label_ for e in ex6.ents])
print([e for e in ex6.ents if e.label_ == 'DATE'])

prep
number
nsubj
was
ROOT
VERB
VBD
[number, 31,800, at, and, 32,300, .]
['nsubj', 'attr', 'prep', 'cc', 'conj', 'punct']
['NN', 'CD', 'IN', 'CC', 'CD', '.']
[31,800, 32,300]
['', 'I', '', '', 'B', '']
[employees, Company, approximately 31,800, December 31, 2016, 32,300, December 31, 2015]
['EMP_NOUN', 'ORG', 'CARDINAL', 'DATE', 'CARDINAL', 'DATE']
[December 31, 2016, December 31, 2015]


In [74]:
displacy.render(ex6, style='dep', jupyter=True, options={'distance': 110})

In [75]:
extract_emp_relations(ex6)

[(Company, employees), (31,800, employees), (number, employees)]

Example 5

### `Emp_num` above and below `EMP_NOUN` in parse tree

One emp_num is a `conj` child of the other emp_num

Desired output: 

`(we, had, 56,400, employees)`

In [76]:
ex7 = nlp("At December 31, 2016 and 2015, we had approximately 56,400 and 66,400 employees, respectively.")

In [77]:
print_df(make_tok_df(ex7))

,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,,At,prep,had,ROOT,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
1,DATE,December,pobj,At,prep,PROPN,NNP,object of preposition,"noun, proper singular"
2,DATE,31,nummod,December,pobj,NUM,CD,None,cardinal number
3,DATE,",",punct,December,pobj,PUNCT,",",punctuation,"punctuation mark, comma"
4,DATE,2016,nummod,December,pobj,NUM,CD,None,cardinal number
5,DATE,and,cc,2016,nummod,CCONJ,CC,coordinating conjunction,"conjunction, coordinating"
6,DATE,2015,conj,2016,nummod,NUM,CD,conjunct,cardinal number
7,,",",punct,had,ROOT,PUNCT,",",punctuation,"punctuation mark, comma"
8,,we,nsubj,had,ROOT,PRON,PRP,nominal subject,"pronoun, personal"
9,,had,ROOT,had,ROOT,VERB,VBD,None,"verb, past tense"


In [78]:
print_doc_info(ex7)

doc is: 
At December 31, 2016 and 2015, we had approximately 56,400 and 66,400 employees, respectively.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,1,"December 31, 2016 and 2015",DATE,December,pobj,object of preposition,At,prep,ADP
1,10,"approximately 56,400",CARDINAL,"56,400",nummod,None,employees,dobj,NOUN
2,13,"66,400",CARDINAL,"66,400",conj,conjunct,"56,400",nummod,NUM
3,14,employees,EMP_NOUN,employees,dobj,direct object,had,ROOT,VERB


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,NP,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,1,December,NP,December,pobj,object of preposition,At,prep,ADP
1,8,we,NP,we,nsubj,nominal subject,had,ROOT,VERB
2,10,"approximately 56,400 and 66,400 employees",NP,employees,dobj,direct object,had,ROOT,VERB


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,CARDINAL,approximately,advmod,"56,400",nummod,ADV,RB,adverbial modifier,adverb
1,CARDINAL,"56,400",nummod,employees,dobj,NUM,CD,None,cardinal number
2,CARDINAL,"66,400",conj,"56,400",nummod,NUM,CD,conjunct,cardinal number


In [79]:
ex7_emp_tok = ex7[14]

ex7_emp_num_tok = ex7[11]

In [80]:
ex7_emp_tok

employees

In [81]:
ex7_emp_tok.dep_

'dobj'

In [82]:
ex7_emp_num_tok

56,400

In [83]:
[c for c in ex7_emp_num_tok.conjuncts]

[66,400]

In [84]:
print('Token children: ')
for w in ex7_emp_num_tok.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token children: 
approximately       child.dep_:advmod
and       child.dep_:cc
66,400       child.dep_:conj


In [85]:
#[w for w in ex7_emp_tok.head.lefts if w.dep_ == 'nsubj']
print('Token:  ' + str(ex7_emp_tok) )
for w in ex7_emp_tok.children:
    if w.dep_ == 'nummod':
        print("Token child: " + str(w) + "  has dep_ == 'nummod'")
        print("       Token.child.i : " + str(w.i))
        print('       child.dep_: ' + str(w.dep_))
        print("       child.tag_: '"+ str(w.tag_) + "'" )
        print("       child.pos_: '"+ str(w.pos_) + "'" )
if ex7_emp_tok.dep_ == 'conj':
    print('Token dep_ is conj.')
    if ex7_emp_tok.head.tag_ == 'CD':
        print("       token.head.tag_ == 'CD' ")
        print("       token.head.i : " + str(ex7_emp_tok.head.i))
    print('Token head is: ' + str(ex7_emp_tok.head) + '       head.dep_:  '+ str(ex7_emp_tok.head.dep_) )
    for i, d in enumerate(ex7_emp_tok.doc.ents):
        if d.label_ == 'DATE':
            print(str(i) + "  " + str(d))
            for tok in ex7_emp_tok.doc[d.start:d.end + 1]:
                if tok.tag_ == 'CD':
                    print("CD tok in date: " + str(tok))
#    if ex7_emp_tok.head.dep_ == ''

print('Token dep_: ' + str(ex7_emp_tok.dep_))
print('Token head: ' + str(ex7_emp_tok.head))
print('       head.dep_: ' + str(ex7_emp_tok.head.dep_))
print("       head.tag_: '"+ str(ex7_emp_tok.head.tag_) + "'" )
print("       head.pos_: '"+ str(ex7_emp_tok.head.pos_) + "'" )
print('Token head lefts: ' )
print('Token conjuncts: ' )
print(str([c for c in ex7_emp_tok.conjuncts]))
for w in ex7_emp_tok.head.rights:
    print(str(w) + '       head.left.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        [print(x) for x in w.lefts]
        [print(s)for s in w.subtree if s.dep_ == 'poss']
print('Token ancestors: ')
for w in ex7_emp_tok.ancestors:
    print(str(w) + '       ancestor.dep_:' + str(w.dep_))
print('Token children: ')
for w in ex7_emp_tok.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token:  employees
Token child: 56,400  has dep_ == 'nummod'
       Token.child.i : 11
       child.dep_: nummod
       child.tag_: 'CD'
       child.pos_: 'NUM'
Token dep_: dobj
Token head: had
       head.dep_: ROOT
       head.tag_: 'VBD'
       head.pos_: 'VERB'
Token head lefts: 
Token conjuncts: 
[]
employees       head.left.dep_:dobj
,       head.left.dep_:punct
respectively       head.left.dep_:advmod
.       head.left.dep_:punct
Token ancestors: 
had       ancestor.dep_:ROOT
Token children: 
56,400       child.dep_:nummod


In [86]:
displacy.render(ex7, style='dep', jupyter=True, options={'distance': 110})

In [87]:
print(ex7_emp_tok.head)
print(ex7_emp_tok.head.dep_)
print(ex7_emp_tok.head.head)
print(ex7_emp_tok.head.head.dep_)
print(ex7_emp_tok.head.head.head)
print(ex7_emp_tok.head.head.head.dep_)
print(ex7_emp_tok.head.head.head.pos_)
print(ex7_emp_tok.head.head.head.tag_)
print([c for c in ex7_emp_tok.head.head.head.children])
print([c.dep_ for c in ex7_emp_tok.head.head.head.children])
print([c.tag_ for c in ex7_emp_tok.head.head.head.children])
print([r for r in ex7_emp_tok.head.head.head.subtree if r.tag_== 'CD'])
print([c.ent_iob_ for c in ex7_emp_tok.head.head.head.children])
print([e for e in ex7.ents])
print([e.label_ for e in ex7.ents])
print([e for e in ex7.ents if e.label_ == 'DATE'])

had
ROOT
had
ROOT
had
ROOT
VERB
VBD
[At, ,, we, employees, ,, respectively, .]
['prep', 'punct', 'nsubj', 'dobj', 'punct', 'advmod', 'punct']
['IN', ',', 'PRP', 'NNS', ',', 'RB', '.']
[31, 2016, 2015, 56,400, 66,400]
['', '', '', 'B', '', '', '']
[December 31, 2016 and 2015, approximately 56,400, 66,400, employees]
['DATE', 'CARDINAL', 'CARDINAL', 'EMP_NOUN']
[December 31, 2016 and 2015]


In [88]:
extract_emp_relations(ex7)

[(we, employees), (56,400, employees)]

### Testing on sentences

In [89]:
test_sents = ["As of September 30, 2016, we employed approximately 7,300 employees world-wide.", 
"As of December 31, 2016, the subsidiaries of AEP had a total of 17,634 employees.", 
"At December 31, 2016 and 2015, we had approximately 56,400 and 66,400 employees, respectively.", 
"At December 31, 2016, we had approximately 9,400 full-time employees.", 
"As of October 29, 2016, we employed approximately 10,000 individuals worldwide.", 
"The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31, 2015.", 
"As of December 31, 2016, we had 1,469 total employees.", 
"ADP employed approximately 57,000 persons as of June 30, 2016.", 
"At December 31, 2016, we employed approximately 26,400 employees.", 
"The Company and its subsidiaries employed 1,562 persons at December 31, 2016, 114 of whom are covered by a collective bargaining agreement with District 10 of the International Association of Machinists.", 
"As of December 31, 2016, the Company had 455 employees, an increase of 17 employees from the prior year end.", 
"As of December 31, 2016, we had approximately 17,500 employees worldwide.", 
"Based in Neenah, Wisconsin, at December 31, 2016, the Company employed approximately 17,500 individuals and had 59 manufacturing facilities.", 
"As of January 31, 2017, we employed 7,683 individuals.", 
"At December 31, 2016, Bio-Rad had approximately 8,250 employees.", 
"As of December 31, 2016, we had approximately 8,500 full-time employees and 600 contractors.", 
"Alcoa's total worldwide employment at the end of 2016 was approximately 14,000 employees in 15 countries.", 
"As of December 31, 2016, we employed approximately 2,100 people.", 
"At December 31, 2016, the Company had approximately 11,500 employees.",
"As of February 23, 2017, we employed approximately 41,000 full-time Team Members and approximately 33,000 part-time Team Members.", 
"As of December 31, 2016, we had 699 full-time employees and 202 temporary employees.", 
"As of December 31, 2016, we had 2,646 employees, 1,581 of whom were pilots."]

In [90]:
for i,t in enumerate(test_sents):
    print('Sentence index: ' + str(i))
    print(extract_emp_relations(nlp(t)))

Sentence index: 0
[(we, employees), (7,300, employees)]
Sentence index: 1
[(AEP, employees), (17,634, employees), (total, employees)]
Sentence index: 2
[(we, employees), (56,400, employees)]
Sentence index: 3
[(we, employees), (9,400, employees)]
Sentence index: 4
[(we, individuals), (10,000, individuals)]
Sentence index: 5
[(Company, employees), (31,800, employees), (number, employees)]
Sentence index: 6
[(we, employees), (1,469, employees)]
Sentence index: 7
[(ADP, persons), (ADP, persons), (57,000, persons)]
Sentence index: 8
[(we, employees), (26,400, employees)]
Sentence index: 9
[(the International Association of Machinists, persons), (its, persons), (Company, persons), (1,562, persons)]
Sentence index: 10
[(Company, employees), (455, employees), (17, employees), (increase, employees)]
Sentence index: 11
[(we, employees), (17,500, employees)]
Sentence index: 12
[(Company, individuals), (17,500, individuals)]
Sentence index: 13
[(we, individuals), (7,683, individuals)]
Sentence in

## Testing with paragraphs

In [12]:
train_para_list = train_df.para_text.tolist()

In [13]:
for sent in nlp(train_para_list[0]).sents:
    print(sent)

As of September 30, 2016, we employed approximately 7,300 employees world-wide.
Approximately 860 of our employees in Mexico, 450 employees in Singapore, and 200 employees in Japan are covered by collective bargaining and other union agreements.


In [14]:
print([(token.text, token.tag_) for token in nlp(list(nlp(train_para_list[0]).sents)[0].text)])

[('As', 'IN'), ('of', 'IN'), ('September', 'NNP'), ('30', 'CD'), (',', ','), ('2016', 'CD'), (',', ','), ('we', 'PRP'), ('employed', 'VBD'), ('approximately', 'RB'), ('7,300', 'CD'), ('employees', 'NNS'), ('world', 'NN'), ('-', 'HYPH'), ('wide', 'RB'), ('.', '.')]


In [16]:
for ent in nlp(list(nlp(train_para_list[0]).sents)[0].text).ents:
    print(ent.text, ent.label_)

September 30, 2016 DATE
approximately 7,300 CARDINAL
employees EMP_NOUN


In [47]:
test_list = ["We are a small company with approximately 61 employees.",
"The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31, 2015. ",
"Total workforce level at December 31, 2016 was approximately 150,500",           
"Currently, the Company and its subsidiaries have an aggregate of 35 employees.",
"we employ only 31 employees", 
"We currently have 21 employees",
"We currently employ 26 full-time employees",
"Including our full and part-time personnel, we estimate that we have \
the equivalent of 12 full time employees.",
"As a REIT, we employ only 31 employees and have a cost-effective \
management structure."
            ]

In [18]:
doc0 = nlp(test_sents[0]) 
doc1 = nlp(test_sents[1]) 
doc2 = nlp(test_sents[2]) 
doc3 = nlp(test_sents[3]) 
doc4 = nlp(test_sents[4]) 

## Matcher testing

In [183]:
emp_ner_regex = re.compile(r"employees|individuals|people|persons|team members", re.I)

emp_flag = lambda text: bool(emp_ner_regex.match(text))

IS_EMP = nlp.vocab.add_flag(emp_flag)

matcher = Matcher(nlp.vocab)

num_emp_pat = [{'ENT_TYPE' : 'CARDINAL', 'TAG' : 'CD'}, {IS_EMP : True}]

matcher.add('NUM_EMP', None, num_emp_pat)
matcher.add('EMP_NOUN', None, [{IS_EMP: True}])

In [184]:
matches1 = matcher(doc1)

In [186]:
for match_id, start, end in matches1:
# create a new Span for each match and use the match_id as the label
    span = Span(doc1, start, end, label=match_id)
    doc1.ents = list(doc1.ents) + [span]  # add span to doc.ents

In [164]:
def add_match_ents(doc, matches):
    """Add matches from Matcher instance to a document's entities."""
    for match_id, start, end in matches:
    # create a new Span for each match and use the match_id as the label
        span = Span(doc, start, end, label=match_id)
        doc.ents = list(doc.ents) + [span]  # add span to doc.ents

In [166]:
matcher = Matcher(nlp.vocab)
matcher.add('NumEmp', None, num_emp_pat)

In [128]:
for match in re.finditer(emp_ner_regex, nlp(test_sents[0]).text):
    start, end = match.span()         # get matched indices
    span = doc.char_span(start, end)  # create Span from indices
    print(span.text)

employees


In [189]:
for match_id, start, end in matches1:
    span = doc1[start:end]
    print(span)
    print(span.sent)

17,634 employees
As of December 31, 2016, the subsidiaries of AEP had a total of 17,634 employees.
